In [9]:
import replicate
import json

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
replicate_api_key = os.getenv("REPLICATE_API_KEY")
client = replicate.Client(api_token=replicate_api_key)

## Configuration


In [11]:
with open('keywords.json') as f:
    data = json.load(f)

In [12]:
model = "snowflake/snowflake-arctic-instruct"
top_k = 50
top_p = 0.9
temperature = 0.9
max_new_tokens = 512
min_new_tokens = 0
stop_sequence = "<|im_end|>"
presence_penalty = 2
frequency_penalty = 0.2
system_prompt = "You're a dataset generator. Your response should be a list of specific keywords that are separated by commas. You have to follow the prompt. You should generate as many keywords as possible"
examples = ', '.join(data)
prompt = f"""You have to generate a lots of techical keywords for a job category. The job category is 'Machine Learning Engineer'. The keywords should be related to the job category and must not repeat the keywords that are these examples: {examples}"""

In [13]:
def generate_keywords():
    global prompt, system_prompt, data
    output = ""
    for event in client.stream(
        "snowflake/snowflake-arctic-instruct",
        input={
            "top_k": top_k,
            "top_p": top_p,
            "prompt": prompt,
            "temperature": temperature,
            "max_new_tokens": max_new_tokens,
            "min_new_tokens": min_new_tokens,
            "stop_sequences": stop_sequence,
            "prompt_template": f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n\n<|im_start|>assistant\n",
            "presence_penalty": presence_penalty,
            "frequency_penalty": frequency_penalty
        },
    ):
        output += str(event)
        print(str(event), end="")
    prompt += ", "
    prompt += output
    number = output.count(",")
    print("There are", number, "keywords generated.")
    for word in output.split(","):
        data.append(word.strip())
    return output

In [14]:
for i in range(5):
    generate_keywords()

 Analytics, C++, Python, Data Analysis, Machine Learning, Software Engineering, Quantitative Analysis, IT InfrastructureThere are 7 keywords generated.
 AI Modeling, Artificial Intelligence, Deep Learning, Neural Networks, Natural Language Processing, Computer Vision, Supervised Learning, Unsupervised Learning, Reinforcement LearningThere are 8 keywords generated.
 Machine Learning, Artificial Intelligence, Deep Learning, Neural Networks, Natural Language Processing, Computer Vision, Supervised Learning, Unsupervised Learning, Reinforcement LearningThere are 8 keywords generated.
 Deep Learning, TensorFlow, Neural Networks, Natural Language Processing (NLP), Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), Long Short-Term Memory (LSTM), Artificial Intelligence (AI), Reinforcement Learning, Generative Adversarial Networks (GAN), Transfer Learning, Image ProcessingThere are 11 keywords generated.
 Artificial Intelligence, Machine Learning, Deep Learning, Neural Netwo

In [15]:
data = list(set(data))

In [16]:
with open("keywords.json", "w") as f:
    json.dump(data, f, indent=4)